In [80]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [83]:
# fake = faker.Faker(locale='ru')

In [84]:
# tinkoff_headers = {
#     'User-Agent': fake.chrome(),
#     'accept-language': 'en-US,en;q=0.9',
#     'pragma': 'no-cache',
#     'Host': 'cfg.tinkoff.org',
#     'Origin': 'https://www.tinkoff.ru',
#     'content-type': 'application/json',
#     'referer': 'https://www.tinkoff.ru'
# }

In [99]:
API_URL = "https://cfg.tinkoff.ru/vacancies/public/api/platform/v1/getVacanciesList"
VACANCY_API_URL = "https://cfg.tinkoff.ru/vacancies/public/api/platform/v2/getVacancy?urlSlug="

In [124]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip()

In [125]:
def build_job_infos(cat='it'):
    search_payload = {
        'category': cat,
        'filters': {},
        'limit': 200
    }
    result = requests.post(API_URL, data=search_payload)
    job_infos = []
    for category in result.json()['response']['items']:
        for job_info in category['items']:
            title = job_info['title']
            url = VACANCY_API_URL + job_info['id']
            job_infos.append((title, url))
    return job_infos

In [126]:
total_jobs = build_job_infos('it') + build_job_infos('back-office')
len(total_jobs)

154

In [127]:
job_dicts = []
for title, url in tqdm(total_jobs):
    result = requests.get(url).json()['response']

    job_dict = {
        'title': title,
        'internal_id' : result['id'],
        'url': url,
        'description': clear_string(result['tasks']),
        'responsibilities': clear_string(result['responsibilities']),
        'qualifications': clear_string(result['requirements']),
        'company': 'Tinkoff',
        'publish_date': parser.parse(result['specialtyInfo']['createdAt'])
    }

    job_dicts.append(job_dict)

    time.sleep(0.2)  # 200 ms

100%|██████████| 154/154 [00:45<00:00,  3.42it/s]


In [128]:
snapshot = pd.DataFrame(job_dicts)
print(len(snapshot))
snapshot.sample(5)

154


,title,internal_id,url,description,responsibilities,qualifications,company,publish_date
88,Продуктовый аналитик в команду Compliance,dc72e9c1-8a2f-4acb-aa35-63763fdb7c35,https://cfg.tinkoff.ru/vacancies/public/api/pl...,Мы ищем аналитика на проект по улучшению проце...,Построение системы метрик в продукте Анализ пр...,Опыт работы в качестве продуктового аналитика ...,Tinkoff,2021-07-09 08:48:59.309000+00:00
22,Data Engineer (Asia),e8e16f32-344a-4882-b3e4-b89fadd395d9,https://cfg.tinkoff.ru/vacancies/public/api/pl...,Тинькофф выходит на международный рынок. И Фил...,Участие в разработке ядра продуктов Data Platf...,Знание Python Знание SQL Знание Spark Будет пл...,Tinkoff,2018-03-27 09:04:23.643000+00:00
1,Ведущий Java-разработчик (Тинькофф Инвестиции),07c306aa-e9c1-43dd-a642-c679656b0f8c,https://cfg.tinkoff.ru/vacancies/public/api/pl...,Увеличиваем мощность разработки в Тинькофф Инв...,Проектирование надежных отказоустойчивых серви...,Опыт программирования на Java от 5 лет Опыт ра...,Tinkoff,2018-03-27 09:04:23.643000+00:00
63,Инженер по тестированию (web),444ca427-e489-4f2e-846b-c96e4a323b1e,https://cfg.tinkoff.ru/vacancies/public/api/pl...,Тинькофф — компания нового поколения: без отде...,Проводить функциональное и автоматизированное ...,Имеешь опыт тестирования фронтенда от 2 лет Им...,Tinkoff,2018-03-27 09:03:49.214000+00:00
4,Golang-разработчик в Tinkoff Asia,3da26517-b3e3-4baf-bdde-77123ca7ba49,https://cfg.tinkoff.ru/vacancies/public/api/pl...,Тинькофф выходит на международный рынок. И Фил...,Разработка нагруженных backend-приложений Учас...,Опыт разработки на golang от 1 года Опыт разра...,Tinkoff,2018-03-27 09:04:23.643000+00:00


In [129]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype                  
---  ------            --------------  -----                  
 0   title             154 non-null    object                 
 1   internal_id       154 non-null    object                 
 2   url               154 non-null    object                 
 3   description       154 non-null    object                 
 4   responsibilities  154 non-null    object                 
 5   qualifications    154 non-null    object                 
 6   company           154 non-null    object                 
 7   publish_date      154 non-null    datetime64[ns, tzutc()]
dtypes: datetime64[ns, tzutc()](1), object(7)
memory usage: 9.8+ KB


In [130]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'27-06-2022'

In [131]:
snapshot.to_csv(f'../data/tinkoff/{current_date}.csv')

In [135]:
requests.get(total_jobs[10][1]).json()['response']

{'id': '789e6068-504b-4278-baab-477a156fd814',
 'status': 'published',
 'title': 'PL/SQL разработчик',
 'specialty': 'Backend',
 'specialtyInfo': {'createdAt': '2018-03-27T09:04:23.643Z',
  'deletedAt': None,
  'id': '4c5c5e87-d961-4c74-a385-d0b25e36cdc5',
  'name': 'Backend',
  'order': 1,
  'urlSlug': 'back-end-razrabotka'},
 'city': 'Любой город',
 'experience': 'Middle',
 'tasks': '<p>Тинькофф — это огромная экосистема. Нашими клиентами уже стали 13 млн человек по всей России. Мы активно развиваемся, появляются новые продукты и сервисы, за которыми стоят разработчики и инженеры. Наши сервисы должны соответствовать высочайшим стандартам надежности и быстродействия. Поэтому сейчас мы ищем разработчика PL/SQL для усиления команд.</p>',
 'requirements': '<ul><li>Отличное знаешь PL/SQL (Oracle 10g и выше)</li><li>Понимаешь архитектуру oracle, основные структуры данных (индексы, таблицы), алгоритмы доступа к данным (index range/full/skip scan) и их соединения</li><li>Знаешь SQL и имеешь 